In [1]:
!pip install transformers datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
!pip install faiss-cpu


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 59.7 MB/s eta 0:00:00


In [3]:
!pip install pandas numpy


In [5]:
import pandas as pd
from datasets import Dataset
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

with open("Input.txt", "r") as f:
    lines = f.readlines()

data = {"text": lines}
df = pd.DataFrame(data)
hf_dataset = Dataset.from_pandas(df)
retriever_model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = retriever_model.encode(hf_dataset['text'], convert_to_tensor=False)
embeddings = np.array(embeddings)

if embeddings.ndim == 2 and embeddings.shape[0] > 0:
    index = faiss.IndexFlatL2(embeddings.shape[1])
    index.add(embeddings)
else:
    raise ValueError("Embeddings are not properly generated.")

doc_map = hf_dataset['text']

def retrieve(query, k=5):
    query_vec = retriever_model.encode([query])[0]
    distances, indices = index.search(np.array([query_vec]), k)
    return [doc_map[i] for i in indices[0]]

generator_model_name = "facebook/bart-large-cnn"
tokenizer = AutoTokenizer.from_pretrained(generator_model_name)
generator = AutoModelForSeq2SeqLM.from_pretrained(generator_model_name)

def generate_response(context, query):
    input_text = f"Context: {context} Query: {query}"
    inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True)
    outputs = generator.generate(**inputs, max_length=150, num_beams=3, early_stopping=True)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

def rag_chatbot(query, k=5):
    retrieved_docs = retrieve(query, k)
    context = " ".join(retrieved_docs)
    response = generate_response(context, query)
    return response

user_query = "What is the content of document 1?"
response = rag_chatbot(user_query)
print(response)


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

 Context: Its a total package. Well packaged, arrived on time, and works as intended. It has everything I need and I couldn't ask for more. All in all, I'm quite satisfied with this purchase. I'm looking forward to using it in the future.
